<a href="https://colab.research.google.com/github/hasbahnah/-MACE-PROJECT-/blob/main/lammps_to_mace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from ase.io import read
import numpy as np

In [2]:


def write_mace_xyz(atoms_list, energies, filename):
    with open(filename, 'w') as f:
        for i, atoms in enumerate(atoms_list):
            # Number of atoms
            f.write(f"{len(atoms)}\n")

            # Header line with energy and lattice
            cell = atoms.get_cell()
            lattice = f"Lattice=\"{cell[0,0]:.8f} {cell[0,1]:.8f} {cell[0,2]:.8f} {cell[1,0]:.8f} {cell[1,1]:.8f} {cell[1,2]:.8f} {cell[2,0]:.8f} {cell[2,1]:.8f} {cell[2,2]:.8f}\""
            f.write(f"TotEnergy={energies[i]:.8f} cutoff=-1.00000000 nneightol=1.20000000 pbc=\"T T T\" {lattice} Properties=species:S:1:pos:R:3:force:R:3:Z:I:1\n")

            positions = atoms.get_positions()

            # Read forces and correct atom types from the dump file
            with open('trajectory.dump', 'r') as f_dump:
                lines = f_dump.readlines()
                natoms = len(atoms)
                forces = np.zeros((natoms, 3))
                atom_types = []  # To store correct atom types

                # Find the correct frame
                frame_start = None
                for j, line in enumerate(lines):
                    if 'ITEM: ATOMS' in line:
                        frame_start = j + 1
                        if i == 0:  # First frame
                            break
                        i -= 1

                # Read forces and atom types for this frame
                for j in range(natoms):
                    line = lines[frame_start + j].split()
                    atom_type = int(line[1])  # LAMMPS atom type (1 or 2)
                    # Convert LAMMPS type to actual atom symbol
                    symbol = 'O' if atom_type == 1 else 'H'  # Type 1 is O, Type 2 is H
                    atom_types.append(symbol)
                    forces[j] = [float(x) for x in line[5:8]]

            # Write atomic data with correct atom types
            for j in range(len(atoms)):
                Z = 8 if atom_types[j] == 'O' else 1
                f.write(f"{atom_types[j]:2s} {positions[j,0]:15.8f} {positions[j,1]:15.8f} {positions[j,2]:15.8f} {forces[j,0]:15.8f} {forces[j,1]:15.8f} {forces[j,2]:15.8f} {Z:3d}\n")

# Read LAMMPS trajectory
print("Reading LAMMPS trajectory...")
atoms = read('trajectory.dump', format='lammps-dump-text', index=':')
print(f"Read {len(atoms)} frames")

print("Reading energies...")
energies = np.loadtxt('energy.dat')
print(f"Read {len(energies)} energy values")

# Debug information
print("\nChecking first frame:")
print("Available arrays:", list(atoms[0].arrays.keys()))

# Write custom extended XYZ
print("\nWriting extended XYZ file...")
write_mace_xyz(atoms, energies, 'traj1.extxyz')
print("Done!")

Reading LAMMPS trajectory...
Read 6 frames
Reading energies...
Read 6 energy values

Checking first frame:
Available arrays: ['numbers', 'positions']

Writing extended XYZ file...
Done!
